# 카메라와 동영상 파일 다루기

* 카메라 영상 & 동영상 파일 처리 공통점 : 연속적인 프레임 영상을 받아와 처리함
* OpenCV의 **cv2.VideoCapture()** 라는 하나의 클래스를 이용해 정지 영상 프레임 받아옴

## 카메라 영상 처리

In [1]:
# cap.get(...) : 현재 열려 있는 카메라 장치 또는 동영상 파일로부터 여러 정보 받아옴

import cv2

cap = cv2.VideoCapture(0) # 0 (index) : 여러 device 중 하나(0번)를 열기 (노트북 캠)
if not cap.isOpened():
    print("Camera open failed!")
    exit()

print(cap)
print('Frame width :', int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
print('Frame height :', int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

<VideoCapture 000002522804C410>
Frame width : 640
Frame height : 480


## 동영상 파일 처리
* 동영상 파일은 고유의 **코덱(codec)**을 이용해 **압축**된 형태로 저장됨 (압축 방법)
* ex. MPEG-4, H.264...
* 카메라 입력 처리 방법과 유사
* But, 길이가 정해져있으므로, **초당 프레임 수(FPS)** 값 가지고 있음
    * FPS 값 고려하지 않을 경우, 동영상이 너무 빠르거나 느리게 재생되는 경우 발생
    * **cv2.CAP_PROP_FPS**

In [3]:
# FPS (Frames Per Second) 값 확인 ==> 고려 안하면 동영상이 너무 빠르거나 느리게 재생

fps = cap.get(cv2.CAP_PROP_FPS) 
print('FPS :', fps)

# FPS 값을 이용하여 매 프레임 사이의 시간 간격 계산 (delay) 
# ex. FPS : 30 ==> 초당 30프레임 ==> 1프레임당 1/30초 ==> 1프레임당 1/30 *1000 ms
# 이 delay 값 : 추후 waitKey() 함수의 인자로 사용 ==> 몇 초마다 프레임 가져올 지

FPS : 30.0


In [2]:
import cv2

cap = cv2.VideoCapture('stopwatch.avi') # 파일명은 *.avi, *.mpg, *.mp4 등의 확장자
if not cap.isOpened():
    print("Video open failed!")
    exit()

print(cap)
print('Frame width :', int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
print('Frame height :', int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print('Frame count :', int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

fps = cap.get(cv2.CAP_PROP_FPS)
print('FPS :', fps)
delay = round(1000 / fps) # 1s == 1000ms


while True:
    ret, frame = cap.read() # 각 frame을 읽어서 (ret (return값) : True or False)
    
    if not ret: # 에러가 없을 경우 if문 안들어가고 아래로 넘어감 (ret : return)
        break
        
    inversed = ~frame # 반전시키고 (inverse : 255 - 원본)
    
    cv2.imshow('frame', frame)
    cv2.imshow('inversed', inversed)
    
    # cv2.waitKey(int)의 인자 : 그 인자만큼 대기했다가 그 인자만큼마다 키 입력 받아옴
    if cv2.waitKey(delay) == 27: # delay 마다 화면에 출력 ==> 모든 프레임 / 27 : esc
        break  # 즉, 각 프레임 사이에 몇 초를 쉬어야 하는지 (==delay!)

cv2.destroyAllWindows() # 모든 창 닫기

<VideoCapture 00000252284DDBB0>
Frame width : 640
Frame height : 480
Frame count : 378
FPS : 30.0


## 동영상 파일 저장하기
* **cv2.VideoWriter()** 클래스 사용
* 프레임을 동영상 파일로 저장하는 기능
* 인자 중 **fourcc** : 4-문자 코드(4-character code)의 약자 (4개의 문자로 구성된 코드)
    * fourcc에 해당하는 **정수** 값은 **cv2.VideoWriter_fourcc()** 함수를 사용하여 생성
    * 동영상 파일의 코덱, 압축방식, 색상 or 픽셀 포맷 등을 정의하는 정수 값.
    * 코덱을 표현하는 네 개의 문자를 묶어서 생성
    > 따라서, cv2.VideoWriter_fourcc()의 return 값 (**fourcc**) : **정수**

In [5]:
(w, h) = (640, 480)

In [7]:
# VideoWriter 클래스의 초기화

fourcc = cv2.VideoWriter_fourcc(*'DIVX') # DicX MPEG-4 코덱 ('X264' : H.264 코덱)

outputVideo = cv2.VideoWriter('output.avi', fourcc, fps, (w,h))
# (저장할 파일 이름, 코덱 종류, 저장할 동영상의 FPS 값, (프레임의 w, h))


# 여기에 write()함수를 사용하여 프레임 한장 한장 추가
# 단, 처음에 지정했던 (w, h) 크기와 프레임 크기가 일치해야 새로 추가(write) 가능!

## 동영상 읽고 처리 후 저장하기 예제
* **cap = cv2.VideoCapture()**
    * cap.**read()**
* **outputVideo = cv2.VideoWriter()**
    * cv2.VideoWriter_fourcc()
    * outputVideo.**write()**

In [6]:
# stopwatch.avi를 읽어서 반전한 영상을 저장하기

import cv2

cap = cv2.VideoCapture('stopwatch.avi')

if not cap.isOpened():
    print('Video open failed!')
    exit()
    
w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
#print(fps) # 30

fourcc = cv2.VideoWriter_fourcc(*'DIVX') # int
delay = round(1000 / fps)


outputVideo = cv2.VideoWriter('output.avi', fourcc, fps, (w,h))
if not outputVideo.isOpened():
    print('File open failed!')
    exit()
    
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    inversed = ~frame
    outputVideo.write(inversed)
    
    cv2.imshow('frame', frame)
    cv2.imshow('inversed', inversed)
    
    if cv2.waitKey(delay) == 27:
        break

cv2.destroyAllWindows()

<hr>

# 다양한 그리기 함수

## 직선 그리기
* 파이썬에서는 img가 numpy array
> cf. **OpenCV**에서는 NumPy와 달리 **(width, height, channel)** 순서!
* line() : 영상 위에 직선 그리기
* arrowedLine() : 화살표 형태의 직선 그리기
    * (인자 중) tipLength : 전체 직선 길이에 대한 화살표 길이의 비율
* drawMarker() : 마커를 그리는 경우 (특정 위치 표시 시)

## (실습) 다양한 그리기 함수 활용
* 수평 방향 직선, 대각선 방향 직선, 수평 방향 화살표, 마커를 그리는 예제
> * **기본 직선 :cv2. line(img, 시작점, 끝점, (B,G,R), 굵기, connectivity)**  
> * **화살표 직선 :cv2. arrowedLine(img, 시작점, 끝점, (B,G,R), 굵기, connectivity, shift, tipLength)**  
> * **마커 :cv2. drawMarker(img, 시작점, 끝점, (B,G,R), 마커 타입)**  

In [14]:
import numpy as np
import cv2 as cv

img = np.full((400, 400, 3), 255, np.uint8) # 255(흰색)으로 채우기 (영상 초기화)

# 수평 방향 직선 그리기 (img, 시작점, 끝점, (B,G,R), 굵기)
cv.line(img, (50, 50), (200, 50), (0, 0, 255)) # (B, G, R) ==> 빨간색 직선
cv.line(img, (50, 100), (200, 100), (255, 0, 255), 3) # 보라색 직선, 굵기 3
cv.line(img, (50, 150), (200, 150), (255, 0, 0), 10) # 파란색 직선, 굵기 10

# 대각선 방향 직선
cv.line(img, (250, 50), (350, 100), (0, 0, 255), 1, cv.LINE_4) # 4방향 연결
cv.line(img, (250, 70), (350, 120), (255, 0, 255), 1, cv.LINE_8) # 8방향 연결
cv.line(img, (250, 90), (350, 140), (255, 0, 0), 1, cv.LINE_AA) #안티앨리어싱

# 수평방향 화살표 직선
cv.arrowedLine(img, (50, 200), (150, 200), (0, 0, 255), 1)
cv.arrowedLine(img, (50, 250), (350, 250), (255, 0, 255), 1)
cv.arrowedLine(img, (50, 300), (350, 300), (255, 0, 0), 1, cv.LINE_8, 0, 0.05)

# 마커
cv.drawMarker(img, (50, 350), (0, 0, 255), cv.MARKER_CROSS)
cv.drawMarker(img, (100, 350), (0, 0, 255), cv.MARKER_TILTED_CROSS)
cv.drawMarker(img, (150, 350), (0, 0, 255), cv.MARKER_STAR)
cv.drawMarker(img, (200, 350), (0, 0, 255), cv.MARKER_DIAMOND)
cv.drawMarker(img, (250, 350), (0, 0, 255), cv.MARKER_SQUARE)
cv.drawMarker(img, (300, 350), (0, 0, 255), cv.MARKER_TRIANGLE_UP)
cv.drawMarker(img, (350, 350), (0, 0, 255), cv.MARKER_TRIANGLE_DOWN)

cv.imshow("img", img)
cv.imwrite('line, arrowedLine, Marker.png', img)
cv.waitKey()
cv.destroyAllWindows()

![img](line,arrowedLine,Marker.png)

## 도형 그리기
* **rectancle()** : 사각형 그리기 ==> 대각 위치의 두 꼭지점 좌표 이용
* **circle()** : 원 그리기 ==> 원의 중심 좌표와 반지름 지정
* **ellipse()** : 타원 그리기 ==> 타원의 중심 좌표와 x축, y축 반지름, 회전 각도 등
    * elipse(img, center, (긴거리, 짧은거리), 회전각도, 시작각, 끝각, 색상, 선.. 등)
* **polylines()** : 임의의 다각형 그리기(2차원 배열에 점들 저장) ==> 닫혀 있는지 유무

## (실습) 도형 그리기
* 다양한 사각형, 원, 타원, 다각형을 화면에 출력

In [2]:
import cv2 as cv
import numpy as np

img = np.full((400, 400, 3), 255, np.uint8)

# 사각형 그리기
cv.rectangle(img, (50, 50), (150, 100), (0, 0, 255), 2) # 2 : 테두리 굵기
cv.rectangle(img, (50, 150), (150, 200), (0, 0, 128), -1) # -1 : 안에 채우기

# 원 그리기
cv.circle(img, (300, 120), 30, (255, 255, 0), -1, cv.LINE_AA) # 안에 채우기
cv.circle(img, (300, 120), 60, (255, 0, 0), 3, cv.LINE_AA)

# 타원 그리기 
# 하늘색 타원 : 0도 ~ 270도까지 안에 채우기 / 20도 회전하여 그리기
cv.ellipse(img, (120, 300), (60, 30), 20, 0, 270, (255, 255, 0), cv.FILLED, cv.LINE_AA)
# 연두색 타원 : 0도 ~ 360도까지 안채우고 선 굵기 : 2 / 20도 회전하여 그리기
cv.ellipse(img, (120, 300), (100, 50), 20, 0, 360, (0, 255, 0), 2, cv.LINE_AA)

# 다각형 그리기
pts = np.array([[250, 250], [300, 250], [300, 300], [350, 300], [350, 350], [250, 350]])
cv.polylines(img, [pts], True, (255, 0, 255), 2) # True : 마지막 두 점 닫기

cv.imshow('img', img)
cv.imwrite('rect,circle,ellipse,polyline.png', img)
cv.waitKey()
cv.destroyAllWindows()

![img](rect,circle,ellipse,polyline.png)

## 문자열 출력하기
* **cv2.putText(img, text, '좌측하단'좌표, 폰트, 폰트크기, 색상, (두께) 등)**
* 콘솔 출력뿐만 아니라, **영상에** 직접 처리 결과 및 추가 정보를 문자열로 출력할 때
* 영문자, 숫자 출력 가능 (한글 출력 불가능)
* (인자 중) bottomLeftOrigin ==> 좌측 하단을 원점으로 할지 유무 (기본적으로 좌측상단)  
  
* **cv2.getTextSize()** : 문자열 출력을 위해 필요하나 사각형 영역 크기 가늠하기 위해

In [5]:
# (실습) 문자열 출력하기

import numpy as np
import cv2 as cv

img = np.full((500, 800, 3), 255, np.uint8)

cv.putText(img, "FONT_HERSHEY_SIMPLEX", (20, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255))
cv.putText(img, "FONT_HERSHEY_PLAIN", (20, 100), cv.FONT_HERSHEY_PLAIN, 1, (0, 0, 255))
cv.putText(img, "FONT_HERSHEY_DUPLEX", (20, 150), cv.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255))
cv.putText(img, "FONT_HERSHEY_COMPLEX", (20, 200), cv.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0))
cv.putText(img, "FONT_HERSHEY_TRIPLEX", (20, 250), cv.FONT_HERSHEY_TRIPLEX, 1, (255, 0, 0))
cv.putText(img, "FONT_HERSHEY_COMPLEX_SMALL", (20, 300), cv.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 0))
cv.putText(img, "FONT_HERSHEY_SCRIPT_SIMPLEX", (20, 350), cv.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (255, 0, 255))
cv.putText(img, "FONT_HERSHEY_SCRIPT_COMPLEX", (20, 400), cv.FONT_HERSHEY_SCRIPT_COMPLEX, 1, (255, 0, 255))
cv.putText(img, "FONT_HERSHEY_COMPLEX | FONT_ITALIC", (20, 450), cv.FONT_HERSHEY_COMPLEX | cv.FONT_ITALIC, 1, (255, 0, 0))

cv.imshow('img', img)
cv.imwrite('text.png', img)
cv.waitKey()
cv.destroyAllWindows()

![img](text.png)

In [10]:
# (실습) 출력할 문자열의 크기를 바탕으로 정중앙에 출력하기

img = np.full((200, 600, 3), 255, np.uint8) # (in OpenCV image) (row, col, channel)

print(img.shape) # (height, width, channel) / (200, 600, 3)

text = "Hello, OpenCV"
fontFace = cv.FONT_HERSHEY_TRIPLEX
fontScale = 2.0 # 폰트 크기 확대/축소 비율
thickness = 1 # 문자열을 그릴 때 사용할 선 두께

# sizeText : 출력할 문자열이 차지할 사각형 영역 
sizeText, _ = cv.getTextSize(text, fontFace, fontScale, thickness) 
print(sizeText) # (481, 43) == (width, height)

# org : img의 크기와 sizeText 정보로 출력할 좌표 계산 : (x, y)==(width, height)
org = ((img.shape[1] - sizeText[0]) // 2, (img.shape[0] + sizeText[1]) // 2)
cv.putText(img, text, org, fontFace, fontScale, (255, 0, 0), thickness)
cv.rectangle(img, org, (org[0] + sizeText[0], org[1] - sizeText[1]), (0, 255, 0), 1)

cv.imshow('img', img)
cv.imwrite('text_center.png', img)
cv.waitKey()
cv.destroyAllWindows()

(200, 600, 3)
(481, 43)


![img](text_size.png)